In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250m; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#################################
#LOADING DATA

In [ ]:
#NEEDED TO PLOT THE CORRECT DATA
data_type="Tracked_Area"

In [ ]:
type1='CL';type2='nonCL'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)
        # print(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
    # print(key)

In [ ]:
#if SE data not available, makes zeroed_out dummy variables in their_place
for key in key_list:
    dummy=ProfileMean(globals()[key])
    dummy[:,0]=np.zeros_like(dummy[:,0])
    dummy=dummy[dummy[:,1]>1]
    globals()[key+f"_SE"]=dummy

In [ ]:
type1='SBZ';type2='nonSBZ'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
    # print(key)

In [ ]:
#if SE data not available, makes zeroed_out dummy variables in their_place
for key in key_list:
    dummy=ProfileMean(globals()[key])
    dummy[:,0]=np.zeros_like(dummy[:,0])
    globals()[key+f"_SE"]=dummy

In [ ]:
type1='ColdPool'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
filePath=dir3+f"{data_type}_"+f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
    # print(key)

In [ ]:
#if SE data not available, makes zeroed_out dummy variables in their_place
for key in key_list:
    dummy=ProfileMean(globals()[key])
    dummy[:,0]=np.zeros_like(dummy[:,0])
    globals()[key+f"_SE"]=dummy

In [ ]:
############################
#PLOTTING

In [ ]:
#LIMITING Y AXIS
limit_y=True
# limit_y=False

In [ ]:
def limit_axes_to_y(ax, y_min=0, y_max=7, buffer_frac=0.1):
    ax.set_ylim(y_min, y_max)

    x_limited = []
    for line in ax.get_lines():
        xdata, ydata = np.array(line.get_xdata()), np.array(line.get_ydata())
        # Mask for y within limits
        y_mask = (ydata >= y_min) & (ydata <= y_max)
        # Apply mask
        x_visible = xdata[y_mask]
        # Remove NaN or Inf from x_visible
        x_visible = x_visible[np.isfinite(x_visible)]
        x_limited.extend(x_visible)
    if len(x_limited) > 0:
        x_limited = np.array(x_limited)
        x_min, x_max = np.min(x_limited), np.max(x_limited)

        if not (np.isfinite(x_min) and np.isfinite(x_max)):
            print("Warning: Non-finite x-limits detected, skipping set_xlim")
            return
        x_range = x_max - x_min

        if x_range == 0:
            buffer = 0.1  # fixed small buffer
        else:
            buffer = buffer_frac * x_range
        ax.set_xlim(x_min - buffer, x_max + buffer)
    else:
        print("Warning: No visible x data within y limits to set xlim")


In [ ]:
def averaged_profiles(profile): 
    out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data
    out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
    return out_var

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
all_cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {all_cloudbase}\n")

In [ ]:
def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
def plot_updraft_profiles(type1, type2, type3, color, ax=None):
    lw=0.8
    # Create new figure and axes if none provided
    if ax is None:
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax = np.ravel(ax)  # Ensure ax is flat list/array

    factor = 1
    switch = 1

    profile_labels = {
        type1: {'label': f"{type3} ({type1})", 'linestyle': 'solid', 'color': color, 'alpha': 0.1, 'hatch': None},
        type2: {'label': f"{type3} ({type2})", 'linestyle': 'dashed', 'color': color, 'alpha': 0.2, 'hatch': '///'}
    }

    profile_types = ['general', 'cloudy']

    for i, prof_type in enumerate(profile_types):
        for t in [type1, type2]:
            array_name = f"{t}_{type3}_profile_array_{prof_type}"
            SE_name = f"{t}_{type3}_profile_array_{prof_type}_SE"
            profile_array = globals()[array_name]
            SE_array = globals()[SE_name]

            profile = averaged_profiles(profile_array)
            SE_profile = SE_array

            ax[i].plot(profile[:, 0], profile[:, 1],
                       color=profile_labels[t]['color'],
                       linestyle=profile_labels[t]['linestyle'],
                       label=profile_labels[t]['label'],lw=lw)

            fill_kwargs = {
                'alpha': profile_labels[t]['alpha'],
                'linewidth': 0.0
            }

            if profile_labels[t]['hatch'] is None:
                fill_kwargs['color'] = profile_labels[t]['color']
            else:
                fill_kwargs['color'] = 'none'
                fill_kwargs['edgecolor'] = profile_labels[t]['color']
                fill_kwargs['hatch'] = profile_labels[t]['hatch']

            # ax[i].fill_betweenx(
            #     profile[:, 1],
            #     np.maximum(profile[:, 0] - factor * SE_profile[:, 0] * switch, 0),
            #     profile[:, 0] + factor * SE_profile[:, 0] * switch,
            #     **fill_kwargs
            # )
            
        ax[i].set_xlabel(r'A ($km^2$)')
        ax[i].set_ylabel('z (km)')
        ax[i].axhline(all_cloudbase, color='purple', linestyle='dashed')
        ax[i].axhline(MeanLFC/1000, color='green', linestyle='dashed')
        ax[i].set_title(f'{prof_type.capitalize()} Profile')
        ax[i].legend()

    if ax is None:
        fix_x_limits(ax)
        fix_y_limits(ax)
        plt.suptitle(f'{type3} {type1} vs {type2} Updrafts Area Profile')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


In [ ]:
#DIFFERENCE PLOT
def average_difference(array1, array2):
    out_var_one=averaged_profiles(array1)
    out_var_two=averaged_profiles(array2)

    #masking out non matches
    second_col_one = out_var_one[:, 1]
    second_col_two = out_var_two[:, 1]
    mask_one = np.isin(second_col_one, second_col_two)
    mask_two = np.isin(second_col_two, second_col_one)
    
    out_var_one = out_var_one.copy()[mask_one]
    out_var_two = out_var_two.copy()[mask_two]
    
    diff=(out_var_one[:,0]-out_var_two[:,0])
    zs=out_var_one[:,1]

    out_profile=np.zeros((len(diff),2))

    out_profile[:,0]=diff;out_profile[:,1]=zs;
    return out_profile

def compute_diff(group1, group2, cat, type1):
    profile1 = globals()[f"{group1}_{cat}_profile_array_{type1}"]
    profile2 = globals()[f"{group2}_{cat}_profile_array_{type1}"]
    globals()[f"{group1}_{group2}_{cat}_diff_{type1}"] = average_difference(profile1, profile2)
    # print(f"{group1}_{group2}_{cat}_diff_{var.lower()}")

def PlotDifference(group1,group2,type1,ax=None):
    ######################
    #Calculating
    # group1='CL';group2='nonCL'
    cats=['ALL','SHALLOW','DEEP']
    for cat in cats:
        compute_diff(group1,group2,cat,type1)

    ######################
    #Plotting
    lw=1
    
    # List of profile arrays and their corresponding labels and x-axis titles
    profiles = [
        (globals()[f"{group1}_{group2}_ALL_diff_{type1}"], 'ALL','black'),
        (globals()[f"{group1}_{group2}_SHALLOW_diff_{type1}"], 'SHALLOW','green'),
        (globals()[f"{group1}_{group2}_DEEP_diff_{type1}"], 'DEEP','blue')
    ]

    # Set up the figure
    if ax==None:
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
    
    # Loop through profiles for CL and nonCL
    for i, (profile, CLlabel,color) in enumerate(profiles):
    
        #CUT BELOW CERTAIN Z
        ax.plot(profile[:, 0], profile[:, -1], color=color, label=CLlabel,lw=lw)
        # apply_scientific_notation([ax])
    
    
    # Add labels and title
    ax.set_xlabel(r"($km^2$)")
    ax.set_ylabel('z (km)')
    ax.grid(True)
    
    ax.axvline(0,linestyle='dashed',color='grey',zorder=-10)
    
    # Show legend
    ax.legend()
    
    #MEAN CLOUD BASE
    ax.axhline(all_cloudbase,color='purple',linestyle='dashed',lw=lw)
    ax.axhline(MeanLFC/1e3,color='green',linestyle='dashed',lw=lw)

    #FIX YLIMIT
    ax.set_ylim(bottom=0)
    #LIMITING YAXIS TO BELOW 7 KM
    if limit_y==True: 
        ax.set_ylim(bottom=0,top=7)
    
    # Set up the main title and adjust layout
    plt.suptitle(f'({group1} minus {group2}) General and Cloudy Tracked Updrafts Area Profile\n')
    ax.set_title(f"{type1[0].upper()+type1[1:]} Updraft Area")


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot two types on the same axes
plot_updraft_profiles('CL', 'nonCL', 'ALL', 'black', ax=ax)
plot_updraft_profiles('CL', 'nonCL', 'SHALLOW', 'green', ax=ax)
plot_updraft_profiles('CL', 'nonCL','DEEP','blue', ax=ax)

for axis in ax:
    #LIMITING YAXIS TO BELOW 7 KM
    if limit_y==True: 
        limit_axes_to_y(axis,y_min=0, y_max=7)   
fix_x_limits(ax)
fix_y_limits(ax)
for axis in ax:
    axis.set_xlim(left=0)
    axis.set_ylim(bottom=0)
 
plt.suptitle('General and Cloudy Tracked Updrafts Area Profile (CL and nonCL)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# #DIFFERENCE PLOT
# group1='CL';group2='nonCL'
# fig, axs = plt.subplots(1, 2, figsize=(12, 6),sharey=True)
# PlotDifference(group1, group2, type1='general', ax=axs[0])
# PlotDifference(group1, group2, type1='cloudy', ax=axs[1])
# plt.tight_layout()

In [ ]:
#######################
#SBZ vs nonSBZ

In [ ]:
############################
#PLOTTING

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot two types on the same axes
plot_updraft_profiles('SBZ', 'nonSBZ', 'ALL', 'black', ax=ax)
plot_updraft_profiles('SBZ', 'nonSBZ', 'SHALLOW', 'green', ax=ax)
plot_updraft_profiles('SBZ', 'nonSBZ','DEEP','blue', ax=ax)

for axis in ax:
    #LIMITING YAXIS TO BELOW 7 KM
    if limit_y==True: 
        limit_axes_to_y(axis,y_min=0, y_max=7)   
fix_x_limits(ax)
fix_y_limits(ax)
for axis in ax:
    axis.set_xlim(left=0)
    axis.set_ylim(bottom=0)
plt.suptitle('General and Cloudy Tracked Updrafts Area Profile (SBZ and nonSBZ)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# #DIFFERENCE PLOT
# group1='SBZ';group2='nonSBZ'
# fig, axs = plt.subplots(1, 2, figsize=(12, 6),sharey=True)
# PlotDifference(group1, group2, type1='general', ax=axs[0])
# PlotDifference(group1, group2, type1='cloudy', ax=axs[1])
# plt.tight_layout()

In [ ]:
#######################
#COLD POOL

In [ ]:
#############################
#PLOTTING

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# # Plot two types on the same axes
# plot_updraft_profiles('SBZ', 'ColdPool', 'ALL', 'black', ax=ax)
# plot_updraft_profiles('SBZ', 'ColdPool', 'SHALLOW', 'green', ax=ax)
# plot_updraft_profiles('SBZ', 'ColdPool', 'DEEP','blue', ax=ax)

# for axis in ax:
#     #LIMITING YAXIS TO BELOW 7 KM
#     if limit_y==True: 
#         limit_axes_to_y(axis,y_min=0, y_max=7)   
# fix_x_limits(ax)
# fix_y_limits(ax)
# for axis in ax:
#     axis.set_xlim(left=0)
#     axis.set_ylim(bottom=0)
# plt.suptitle('General and Cloudy Tracked Updrafts Area Profile (SBZ and ColdPool)')
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()


In [ ]:
# #DIFFERENCE PLOT
# group1='SBZ';group2='ColdPool'
# fig, axs = plt.subplots(1, 2, figsize=(12, 6),sharey=True)
# PlotDifference(group1, group2, type1='general', ax=axs[0])
# PlotDifference(group1, group2, type1='cloudy', ax=axs[1])
# plt.tight_layout()